In [6]:
import pandas as pd
import os 

cmv_id = pd.read_csv('/home/daria/A02/common/cmv_pv.txt', sep=',')
cmv_id= cmv_id.drop(cmv_id.index[1000:])                                 #Топ 1000
met_data = pd.read_csv('/home/daria/A02/common/metadata.txt', sep='\t', index_col = False)

met_cmv = dict(zip(list(met_data['file_name']), list(met_data['cmv'])))  #Словарь "название образца" : "CMV"  
met_hla = dict(zip(list(met_data['file_name']), list(met_data['hla'])))  #Словарь "название образца" : "список из HLA"
cdr3 = list(cmv_id['cdr3aa'])                                            #Список ID CDR3a

df_col = [0]*len(cdr3)

#Это мой датафрейм
df = pd.DataFrame({'CMV+_HLA-B*07' : df_col, 'CMV+_other' : df_col, 'CMV-_HLA-B*07' : df_col, 'CMV-_other' : df_col}, 
                  index = cdr3)
df.index.name='cdr3aa'
#Это мой датафрейм

def read_col(file, n):                                                         #открывает файл, генерирует список из данных n-ого столбца    
    result = []                                     
    with open(file) as infile:
        for line in infile:
            result.append((line.split()[n]))
    return result    

def IntersLst(first, second):                                                   #возвращиет пересечение двух списков
    in_lst =  list(set(first) & set(second))
    return in_lst

#Это HLA
hla = "HLA-B*07"                                     
#Это HLA

os.chdir("/home/daria/A02/samples")

for filename in os.listdir():
    sample = read_col(filename, 3)
    found = IntersLst(cdr3, sample)        
    if met_cmv[filename]== '+':
        if hla in list(met_hla[filename].split(',')):
            df.loc[found, 'CMV+_HLA-B*07'] += 1
        else:
            df.loc[found, 'CMV+_other'] += 1
    elif met_cmv[filename]== '-':
        if hla in list(met_hla[filename].split(',')):
             df.loc[found, 'CMV-_HLA-B*07'] += 1
        else: 
            df.loc[found, 'CMV-_other'] += 1

df.to_csv('cmv_hela.txt', sep = '\t')


In [ ]:
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.multitest as smt

cmv_hela = pd.read_csv('/home/daria/A02/common/cmv_hela.txt', sep=',') #Открываем cmv_hela.txt
cdr3 = list(cmv_hela['cdr3aa'])                                        #Список из ID  
 
#Это мой датафрейм
df_col = [0]*len(cdr3)
df_pv = pd.DataFrame({'odds_ratio' : df_col, 'p-value' : df_col, 'p-value_adjust' : df_col}, index = cdr3)
df.index
#Это мой датафрейм


for item in cdr3:                                                              #Odds ratio, Fisher test
    a =  cmv_hela.loc[item, 'CMV+_HLA-B*07']
    b = cmv_hela.loc[item, 'CMV+_other']
    c = cmv_hela.loc[item, 'CMV-_HLA-B*07']
    d = cmv_hela.loc[item, 'CMV-_other']
    lst = [[a,b],[c,d]]
    oddsratio, pvalue = stats.fisher_exact(lst)
    df_pv.loc[item, 'odds_ratio'] = oddsratio
    df_pv.loc[item, 'p-value'] =  pvalue

    
    
